In [1]:
import chromadb

In [4]:
# chroma_client = chromadb.Client()

chroma_client = chromadb.PersistentClient(path="../data/chroma_dbs/")

In [5]:
collection = chroma_client.create_collection(name="arxiv_collection")

In [6]:
collection.add(
    # embeddings=[[1.2, 2.3, 4.5], [6.7, 8.2, 9.2]], # could add embed if they are already computed!
    documents=["This is a document", "This is another document"],
    metadatas=[{"source": "my_source"}, {"source": "my_source"}],
    ids=["id1", "id2"]
)

In [7]:
results = collection.query(
    query_texts=["This is a query document"],
    n_results=2
)

In [ ]:
results